In [1]:
!pip install kaggle
import kaggle



In [2]:
from kaggle.api.kaggle_api_extended import KaggleApi

In [3]:
api = KaggleApi()
api.authenticate()

In [4]:
api.dataset_download_cli('jangedoo/utkface-new')

Dataset URL: https://www.kaggle.com/datasets/jangedoo/utkface-new
License(s): copyright-authors


In [5]:
import zipfile

In [6]:
with zipfile.ZipFile('utkface-new.zip', 'r') as zipref:
    zipref.extractall()

In [ ]:
import os
import cv2
import numpy as np

# Define the path to the cropped images
data_dir = "C:\VS CODE\ML\utkface_aligned_cropped\crop_part1"

# Function to load and process images
def load_images_from_folder(data_dir):
    images = []
    labels = []

    for filename in os.listdir(data_dir):
        if filename.endswith(".jpg"):  # Assuming the images are in JPG format
            # Construct the full file path
            file_path = os.path.join(data_dir, filename)

            # Load the image using OpenCV (adjust if you need different processing)
            image = cv2.imread(file_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB

            # Extract the age and ethnicity from the filename (if applicable)
            age = int(filename.split("_")[0])  # Assuming the age is the first element in the filename

            # Append the image and label (age) to the lists
            images.append(image)
            labels.append(age)

    # Convert lists to numpy arrays for use in ML models
    images = np.array(images)
    labels = np.array(labels)

    return images, labels

# Load images and labels
images, labels = load_images_from_folder(data_dir)
print(f"Loaded {len(images)} images with corresponding labels.")
